**Program**: 1b_NSCH_state_data<br>
**Class**: Fall 2025, Machine Learning, Project<br>
**Member**: Vanessa Thorsten<br>
**Description**: This program reads in the NSCH database and calculates weighted proportions rolled up to state-level.<br>
**Outputs**: NSCH_state.csv file of the state-level data
  
**Program History/Modifications**:<br>
09/04/2025    Initial Version


In [1]:
import pandas as pd

In [2]:
NSCH_df = pd.read_csv("NSCH_fam.csv")
NSCH_df.head()

,Unnamed: 0,FIPSST,STATE,STATE_NAME,STRATUM,HHID,FORMTYPE,TOTKIDS_R,FWC,SC_AGE_YEARS,...,MedDentCare_23,K4Q01,GOWHENSICK,UsualSck_23,smAdeqIns_23,smForgoneHC_23,nomAnxietyDep12to17_23,nomFlrish6mto5_23,nomFlrish6to17_23,ScreenTime_23
0,0,6,CA,California,1,23043707,2,1,1318.476840,11,...,1.0,2.0,NaN,2.0,1.0,2.0,NaN,90.0,2.0,4.0
1,1,6,CA,California,1,23120547,3,2,978.499881,14,...,1.0,1.0,1.0,1.0,1.0,2.0,2.0,90.0,1.0,5.0
2,2,6,CA,California,1,23197456,3,1,904.191765,17,...,1.0,1.0,NaN,NaN,1.0,2.0,2.0,90.0,1.0,4.0
3,3,6,CA,California,1,23197458,3,2,1092.097256,12,...,2.0,1.0,1.0,1.0,1.0,2.0,2.0,90.0,1.0,4.0
4,4,6,CA,California,1,23235909,2,2,586.387870,10,...,2.0,2.0,NaN,2.0,2.0,2.0,NaN,90.0,1.0,5.0


In [3]:
NSCH_df['nomChHlthSt_23'].value_counts()

nomChHlthSt_23
1.0    49701
2.0     4458
Name: count, dtype: int64

In [4]:
# Group and sum
summary = NSCH_df.groupby('nomChHlthSt_23')['FWC'].sum()

# Format with commas and no scientific notation
summary_formatted = summary.apply(lambda x: f"{x:,.0f}")
print(summary_formatted)

nomChHlthSt_23
1.0    63,571,702
2.0     6,966,674
Name: FWC, dtype: object


In [5]:
#Check weighted counts for one of the variables
weighted_counts = pd.crosstab(NSCH_df['STATE'], NSCH_df['nomChHlthSt_23'], 
                              values=NSCH_df['FWC'], aggfunc='sum')

formatted_counts = weighted_counts.applymap(lambda x: f"{x:,.0f}")
print(formatted_counts)

nomChHlthSt_23        1.0      2.0
STATE                             
AK                156,441   12,773
AL                985,668  106,826
AR                619,428   57,317
AZ              1,359,120  186,787
CA              7,225,693  948,677
CO              1,093,001   89,815
CT                665,187   60,758
DC                110,304   11,310
DE                183,314   19,638
FL              3,766,775  439,656
GA              2,195,007  244,541
HI                256,656   25,517
IA                649,488   58,938
ID                428,500   20,743
IL              2,483,690  175,169
IN              1,407,614  123,000
KS                617,976   52,879
KY                887,763   98,466
LA                914,912  114,145
MA              1,190,684  128,998
MD              1,229,271   98,226
ME                219,116   23,479
MI              1,828,652  221,695
MN              1,174,652   88,567
MO              1,158,832  185,342
MS                574,142   79,251
MT                20

C:\Users\vromm\AppData\Local\Temp\ipykernel_12260\400235535.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  formatted_counts = weighted_counts.applymap(lambda x: f"{x:,.0f}")


In [6]:
import math

cols_to_prop = ['nomChHlthSt_23', 'PrevMed_23', 'K4Q01', 'smAdeqIns_23']
total_df = pd.read_csv('State_FIPS.csv',usecols=['STATE'])
total_df = total_df.sort_values(by='STATE')
frames = []
for col in cols_to_prop:
    filename = col + "_proportion_df.csv"
    prop_table = pd.crosstab(NSCH_df['STATE'], NSCH_df[col], values=NSCH_df['FWC'], aggfunc='sum').round(1)
    row_sums = prop_table.sum(axis=1)
    for i in prop_table.columns:
        prop_table[str(col) + '_' + str(i) + '_proportion'] = round(prop_table[i]/row_sums,4)
        prop_table.rename(columns ={i : str(col) + "_" + str(i)}, inplace=True)
    
    frames.append(prop_table)

for df in frames:
    total_df = pd.merge(total_df, df, on="STATE", how="left")
total_df.to_csv("NSCH_state.csv")

In [7]:
total_df.head()

,STATE,nomChHlthSt_23_1.0,nomChHlthSt_23_2.0,nomChHlthSt_23_1.0_proportion,nomChHlthSt_23_2.0_proportion,PrevMed_23_1.0,PrevMed_23_2.0,PrevMed_23_1.0_proportion,PrevMed_23_2.0_proportion,K4Q01_1.0,K4Q01_2.0,K4Q01_1.0_proportion,K4Q01_2.0_proportion,smAdeqIns_23_1.0,smAdeqIns_23_2.0,smAdeqIns_23_1.0_proportion,smAdeqIns_23_2.0_proportion
0,AK,156441.3,12772.5,0.9245,0.0755,127964.3,41249.5,0.7562,0.2438,145190.5,24023.3,0.8580,0.1420,107292.0,61921.9,0.6341,0.3659
1,AL,985667.6,106826.1,0.9022,0.0978,867311.2,225182.5,0.7939,0.2061,888660.2,203833.5,0.8134,0.1866,808852.4,283641.3,0.7404,0.2596
2,AR,619428.4,57316.6,0.9153,0.0847,510951.7,165793.2,0.7550,0.2450,531663.1,145081.8,0.7856,0.2144,463003.3,213741.6,0.6842,0.3158
3,AZ,1359119.5,186787.0,0.8792,0.1208,1174407.6,371498.9,0.7597,0.2403,1144031.7,401874.9,0.7400,0.2600,964676.5,581230.0,0.6240,0.3760
4,CA,7225692.6,948677.3,0.8839,0.1161,6128100.8,2046269.0,0.7497,0.2503,5793569.6,2380800.3,0.7087,0.2913,5765790.6,2408579.3,0.7053,0.2947
